# APS ISN Diffractometer

Model of the ISN diffractometer at APS 19ID. This is a 3-axis diffractometer,
based on the *psic* geometry described by Hoydoo Yoo (*J Appl Cryst*, 1999,
**32**, 614-623). This diffractometer geometry is very similar to the TARDIS
diffractometer at NSLS-II.

Some *psic* axes do not exist in *ISN*; they are fixed at zero. The ISN
diffractometer renames the {math}`\nu` and {math}`\delta` axes:

psic axis | ISN axis | range
--- | --- | ---
mu | mu | {math}`\pm 180`
eta | eta | fixed at 0
chi | chi | fixed at 0
phi | phi | fixed at 0
nu | yaw | {math}`\pm 180`
delta | pitch | {math}`\pm 180`

ISN has an additional `radius` motion with range 0-1000 (mm). This axis is not
part of the defined *psic* geometry.

The *E6C* geometry (see *hkl_soleil, E6C* geometry in the
[tables](../diffractometers.rst)) has a `lifting_detector_mu` mode that keeps
`eta` (E6C calls this axis `omega`), `chi`, and `phi` fixed.  This is the only
E6C mode using the `hkl` engine where none of these axes are listed as
*writeable(s)*.

In `hkl_soleil`, this is the *E6C* geometry.

![*psic* drawing](../_static/psic.png)

Here, we setup a simulated ISN diffractometer and orient a GaAs crystal using two orientation reflections.

## Create the Simulated ISN Diffractometer

The simulator defines all the motor axes using
[`ophyd.SoftPositioner`](https://blueskyproject.io/ophyd/user/reference/positioners.html#softpositioner).
The six axes are specified in the order expected by the E6C geometry.  The
additional `radius` axis is added to the list after all the expected axes.
Specified in this order, *hklpy2* will use the first axes to compute {math}`h`,
{math}`k`, & {math}`l` and ignore any remaining *reals*.

In [1]:
import hklpy2
import numpy as np

np.set_printoptions(precision=3, suppress=True)

psic = hklpy2.creator(
    name="psic",
    geometry="E6C",
    solver="hkl_soleil",
    solver_kwargs={"engine": "hkl"},
    reals="mu eta chi phi yaw pitch radius".split(),
) 

Configure some basic operating parameters.
Length units are *angstrom*, angles are degrees, and energy is keV.

In [ ]:
psic.beam.wavelength.put(12.3984 / 20)
psic.yaw.move(40)
psic.mu.move(psic.yaw.position/2)
psic.radius.egu = "mm"
psic.radius._limits = 0, 1000
psic.radius.move(800)
psic.core.mode = "lifting_detector_mu"

Print a brief summary of the ISN.

In [3]:
psic.wh()
psic.core.solver

wavelength=0.6199
pseudos: h=0, k=1.1034, l=0
reals: mu=20.0, eta=0, chi=0, phi=0, yaw=40, pitch=0
auxiliaries: radius=800


HklSolver(name='hkl_soleil', version='5.1.2', geometry='E6C', engine_name='hkl', mode='lifting_detector_mu')

## Add a sample

In [4]:
psic.core.add_sample("GaAs", 5.75)

Sample(name='GaAs', lattice=Lattice(a=5.75, system='cubic'))

## Orient the sample

Add two observed reflections and the motor positions associated with those
{math}`hkl` values.

We specify the motor positions in the order expected by E6C.  We don't need to
specify the axis names.

In [5]:
r001 = psic.core.add_reflection(
    (0, 0, 1),
    (6.18 / 2, 0, 0, 0, 6.18, 0),
    name="r001",
)
r100 = psic.core.add_reflection(
    (1, 0, 0),
    (6.18 / 2, 0, 90, 0, 6.18, 0),
    name="r100",
)

Print the two reflections.

In [6]:
print(f"{r001=}")
print(f"{r100=}")

r001=Reflection(name='r001', h=0, k=0, l=1)
r100=Reflection(name='r100', h=1, k=0, l=0)


Calculate the {math}`UB` (orientation) matrix from the two orienting reflections.

In [7]:
psic.core.calc_UB(r001, r100)

[[-0.0, -1.092727879509, -0.0],
 [-0.0, -0.0, 1.092727879509],
 [-1.092727879509, 0.0, 0.0]]

## Compute *hkl* from motor positions

Just **calculate** the *hkl* positions. These commands do not move the motors.

That is: `pseudos = inverse(reals)`.  Specify the motor positions in the
expected order.

In [8]:
print(f"{psic.inverse(6.18 / 2, 0, 0, 0, 6.18, 0)=}")
print(f"{psic.inverse(6.18 / 2, 0, 90, 0, 6.18, 0)=}")

psic.inverse(6.18 / 2, 0, 0, 0, 6.18, 0)=Hklpy2DiffractometerPseudoPos(h=0, k=0, l=1.0)
psic.inverse(6.18 / 2, 0, 90, 0, 6.18, 0)=Hklpy2DiffractometerPseudoPos(h=1.0, k=0, l=0)


## Compute motor positions from *hkl*

Just **calculate** the motor positions. These commands do not move the motors.

That is: `reals = forward(pseudos)`.

### List of solutions

First, use the `.core.forward()` method to show the **full list** of possible motor
positions.  Note that the `forward()` calculation produces a list of possible
solutions.  

The (0,0,1) reflection should match the first orienting reflection.

The list may be empty, indicating no solutions were found, such as
the case for the (1,0,0) reflection which needs `chi=90`.

In [9]:
print(f"{psic.core.forward((0, 0, 1))=}")
print(f"{psic.core.forward((1, 0, 0))=}")

psic.core.forward((0, 0, 1))=[Hklpy2DiffractometerRealPos(mu=3.0901, eta=0, chi=0, phi=0, yaw=6.1802, pitch=0), Hklpy2DiffractometerRealPos(mu=176.9099, eta=0, chi=0, phi=0, yaw=-6.1802, pitch=0), Hklpy2DiffractometerRealPos(mu=3.0901, eta=0, chi=0, phi=0, yaw=-173.8198, pitch=-180.0), Hklpy2DiffractometerRealPos(mu=3.0901, eta=0, chi=0, phi=0, yaw=-173.8198, pitch=180.0), Hklpy2DiffractometerRealPos(mu=176.9099, eta=0, chi=0, phi=0, yaw=173.8198, pitch=-180.0), Hklpy2DiffractometerRealPos(mu=176.9099, eta=0, chi=0, phi=0, yaw=173.8198, pitch=180.0)]
psic.core.forward((1, 0, 0))=[]


### Default solution

Next, use the diffractometer's `forward()` method (the one used when moving and
scanning the diffractometer position).  This method, by default setting, returns
the first solution returned from the `.core.forward()` method.  A
`NoForwardSolutions` exception is raised if the list of solutions is empty.

In [10]:
print(f"{psic.forward((0, 0, 1))=}")
try:
    print(f"{psic.forward((1, 0, 0))=}")  # unreachable, chi=90
except hklpy2.misc.NoForwardSolutions as exc:
    print(f"Cannot compute forward((1,0,0)): {exc}")

psic.forward((0, 0, 1))=Hklpy2DiffractometerRealPos(mu=3.0901, eta=0, chi=0, phi=0, yaw=6.1802, pitch=0)
Cannot compute forward((1,0,0)): No solutions.


The `forward()` calculation computes new values for the *writeable* axes, using
the current positions of the remaining axes.

Unlike the physical diffractometer, the simulator can be positioned to the
unreachable `chi=90` position to check the (1,0,0) position. With this
adjustment, the computed (1,0,0) reflection should match the second orienting
reflection.

In [11]:
psic.chi.move(90)  # unreachable: ISN chi axis won't move for ISN diffractometer, but we can simulate it
print(f"{psic.forward((1, 0, 0))=}")
psic.chi.move(0)  # put it back

psic.forward((1, 0, 0))=Hklpy2DiffractometerRealPos(mu=3.0901, eta=0, chi=90.0, phi=0, yaw=6.1802, pitch=0)


MoveStatus(done=True, pos=psic_chi, elapsed=0.0, success=True, settle_time=0.0)

### Table of solutions

In [12]:
from hklpy2.user import *

set_diffractometer(psic)
cahkl_table((0, 0, 1), (0, 1, 1), (1,1,1))

======= = ======== === === === ========= =========
(hkl)   # mu       eta chi phi yaw       pitch    
======= = ======== === === === ========= =========
(0 0 1) 1 3.0901   0   0   0   6.1802    0        
(0 0 1) 2 176.9099 0   0   0   -6.1802   0        
(0 0 1) 3 3.0901   0   0   0   -173.8198 -180     
(0 0 1) 4 3.0901   0   0   0   -173.8198 180      
(0 0 1) 5 176.9099 0   0   0   173.8198  -180     
(0 0 1) 6 176.9099 0   0   0   173.8198  180      
(0 1 1) 1 -40.6278 0   0   0   8.7443    0        
(0 1 1) 2 -40.6278 0   0   0   -171.2557 -180     
(0 1 1) 3 -40.6278 0   0   0   -171.2557 180      
(1 1 1) 1 -38.4337 0   0   0   8.7636    -6.1892  
(1 1 1) 2 -38.4337 0   0   0   -171.2364 -173.8108
======= = ======== === === === ========= =========



## Save the orientation to a file

In [13]:
psic.export(
    "dev_aps-isn.yml",
    comment="APS ISN diffractometer with oriented GaAs sample",
)

Show that configuration.

In [14]:
%pycat dev_aps-isn.yml

#hklpy2 configuration file

_header:
  datetime: '2025-11-19 13:08:07.152236'
  hklpy2_version: 0.2.1.dev5+gbbdb58469
  python_class: Hklpy2Diffractometer
  file: dev_aps-isn.yml
  comment: APS ISN diffractometer with oriented GaAs sample
name: psic
axes:
  pseudo_axes:
  - h
  - k
  - l
  real_axes:
  - mu
  - eta
  - chi
  - phi
  - yaw
  - pitch
  axes_xref:
    h: h
    k: k
    l: l
    mu: mu
    eta: omega
    chi: chi
    phi: phi
    yaw: gamma
    pitch: delta
  extra_axes:
    h2: 0
    k2: 0
    l2: 0
    psi: 0
digits: 4
sample_name: GaAs
samples:
  sample:
    name: sample
    lattice:
      a: 1
      b: 1
      c: 1
      alpha: 90.0
      beta: 90.0
      gamma: 90.0
      digits: 4
      angle_units: degrees
      length_units: angstrom
    reflections: {}
    reflections_order: []
    U:
    - - 1
      - 0
      - 0
    - - 0
      - 1
      - 0
    - - 0
      - 0
      - 1
    UB:
    - - 6.283185307179586
      - 0.0
      - 0.0
    - - 0.0
      - 6.283185307179

In [15]:
psic.move(0, 1, 1)
psic.wh()

wavelength=0.6199
pseudos: h=0, k=1.0, l=1.0
reals: mu=-40.6278, eta=0, chi=0, phi=0, yaw=8.7443, pitch=0
auxiliaries: radius=800


## (0kl) scan

TODO:

- scan from (001) to (011) and plot mu & yaw

## Azimuthal psi scan

TODO:

- azimuthal psi scan about {math}`hkl=(011)` & {math}`hkl_2 = (111)`